In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 베이스 모델 먼저 로드
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",   # GPU 자동 배치
    torch_dtype="auto",   # 메모리 절약을 위해 fp16/bf16 자동 선택,
    cache_dir="/workspace/Weight_compression/Wparam_dataset/hf_model/cache"
)

# # 2. LoRA 어댑터 로드
# lora_model_id = "Lots-of-LoRAs/Mistral-7B-Instruct-v0.2-4b-r16-task581"

# model = PeftModel.from_pretrained(model, lora_model_id)

# # 3. 추론 예시
# text = "Explain the difference between supervised and unsupervised learning."
# inputs = tokenizer(text, return_tensors="pt").to(model.device)

# outputs = model.generate(
#     **inputs,
#     max_new_tokens=128,
#     temperature=0.7,
#     top_p=0.9
# )

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [10]:
from huggingface_hub import snapshot_download

repo_id = "Lots-of-LoRAs/Mistral-7B-Instruct-v0.2-4b-r16-task581"

# 원하는 저장 위치 지정
cache_dir = "/workspace/Weight_compression/Wparam_dataset/hf_model/lora/snapshots"

local_dir = snapshot_download(
    repo_id=repo_id,
    cache_dir=cache_dir,    # 기본 ~/.cache 대신 이 폴더에 저장
    local_dir_use_symlinks=False,  # 실제 파일 복사
    resume_download=True    # 중간에 끊겨도 이어받기
)

print("LoRA repo downloaded to:", local_dir)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 4 files: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]

LoRA repo downloaded to: /workspace/Weight_compression/Wparam_dataset/hf_model/lora/snapshots/models--Lots-of-LoRAs--Mistral-7B-Instruct-v0.2-4b-r16-task581/snapshots/501505479d290bb73a578c1994c4af596e370d46


In [3]:
from safetensors.torch import load_file

# LoRA 어댑터 저장 경로
# adapter_path = "/workspace/Weight_compression/Wparam_dataset/lora/snapshots/models--Lots-of-LoRAs--Mistral-7B-Instruct-v0.2-4b-r16-task581/snapshots/501505479d290bb73a578c1994c4af596e370d46/adapter_model.safetensors"
adapter_path = "/workspace/Weight_compression/text-to-lora/train_outputs/sft/oracle_lora/['lol_033']_20250830-161732_edZecUPd/adapter_model.safetensors"
state_dict = load_file(adapter_path)

# 키와 shape 확인
for k, v in list(state_dict.items())[:20]:  # 처음 20개만 출력
    print(k, v.shape)


base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight torch.Size([4096, 8])
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight torch.Size([1024, 8])
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight torch.Size([4096, 8])
base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight torch.Size([1024, 8])
base_model.model.model.layers.10.self_attn.q_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.10.self_attn.q_proj.lora_B.weight torch.Size([4096, 8])
base_model.model.model.layers.10.self_attn.v_proj.lora_A.weight torch.Size([8, 4096])
base_model.model.model.layers.10.self_attn.v_proj.lora_B.weigh

In [2]:
training_task_ids = [
    742,1198,717,705,275,636,84,1711,140,1448,453,1207,734,298,587,703,147,564,1341,201,
    890,908,1428,325,1669,246,357,1518,1148,1605,867,209,751,161,105,645,442,75,269,1568,
    834,1603,685,83,390,750,1631,1529,746,1217,725,889,492,620,294,641,318,846,316,1188,
    629,770,1482,499,955,719,723,87,211,901,1483,89,627,153,1342,828,64,1387,400,1294,243,
    1572,1151,574,428,366,926,1503,130,515,151,619,1562,632,966,605,1487,707,1379,1489,
    1567,1384,1404,691,728,219,964,1509,582,455,963,382,859,1393,1565,1720,670,689,324,
    1420,618,625,377,929,296,852,1332,1444,850,708,1292,110,155,429,245,137,1566,1146,924,
    22,118,687,1167,380,33,1502,865,181,228,698,956,732,721,370,1400,1199,1606,288,1670,
    207,1206,457,1308,1310,874,1541,1609,210,1318,365,755,123,1316,1378,475,903,70,720,
    67,1564,270,167,1504,178,277,1315,1434,192,1157,672,563,714,1212,495,1583,753,343,
    1427,92,1285,333,329,398,157,74,1506,697,285,393,1147,1585,648,353,1431,148,585,81,
    477,1582,355,381,633,93,722,566,1152,1452,1286,925,1703,833,1210,679,1203,1089,389,
    378,637,101,1355,640,344,1190,733,107,1209,505,1385,1135,328,609,413,72,1425,1451,713,
    642,1321,454,504,696,1429,1645,431,1317,131,675,1158,1325,1216,1347,1328,630,460,
    1665,1154,300,617,1508,628,600,1380,489,905,65,44,69,356,403,590,577,819,1721,351,
    904,244,1712,683,1197,1592,274,278,1534,891,694,497,488,144,1722,1728,291,284,1288,
    128,580,616,1401,716,1421,1656,1311,516,593,138,1186,1326,119,108,584,388,45,821,1581,
    695,596,568,85,1495,927,1453,1201,923,1204,1510,754,1403,1192,565,146,666,50,704,934,
    579,1196,267,206,936,323,494,461,1409,1313,76,686,740,34,1211,113,280,1551,116,518,
    1520,79,513,1590,1713,1386,63,183,1447,671,68,858,699,1593,700,1607,121,190,1168,195,
    1723,1449,363,1419,1398,893,326,1194,102,145,1338,391,176,319,359,856,1729,761,1320,
    1596,1601,615,1283,638,607,692,588,129,1200,1486,1406,583,94,1731,308,664,163,127,
    736,379,330,1599,875,1214,933,1319,456,1189,1657,517,385,472,47,1533,249,95,1479,724,
    1418,507,43,1394,149,1156,1598,322,1714,727,80,1704,77,1390,667,1724,1088,1087,550,
    892,674,1389,509,346,769,1391,1322,341,290,335,879,861,594,66,162,1584,1622,247
]

# Validation task ids (11개)
validation_task_ids = [
    35,39,1557,202,304,362,614,701,706,710,726
]
len(training_task_ids), len(validation_task_ids)  # (500, 11)

(489, 11)